In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
import numpy as np
import pandas as pd
from data_util import *

independents = ["dew", "temp", "press", "wind_direction", "wind_speed", "snow", "rain"]
dependent = "pollution"

# Global param
n_steps = 128
window_size = 8
n_variables = len(independents)
samples_size = 100

df = load_data_set_bejin()
x_scaler, y_scaler = get_xy_scalers(df, independents, dependent)
df[independents] = x_scaler.transform(df[independents].values)
df[dependent] = y_scaler.transform(df[dependent].values.reshape(-1, 1))

In [3]:
from random import shuffle

indices = [i for i in range(len(df) - 129)]
train, test = np.split(indices, [int(len(indices)*0.8)])

X = []
y = []
for index in train:
    X.append(df.iloc[index:index+n_steps][independents].values)
    y.append(df.iloc[index+n_steps][dependent])
    
X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)


X_test = []
y_test = []
for index in test:
    X_test.append(df.iloc[index:index+n_steps][independents].values)
    y_test.append(df.iloc[index+n_steps][dependent])
    
X_test = np.array(X_test)
y_test = np.array(y_test)
print(X_test.shape, y_test.shape)

(34936, 128, 7) (34936,)
(8735, 128, 7) (8735,)


In [4]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.optimizers import Adam
import tensorflow as tf
model = tf.keras.models.load_model("data/lstm_mts_128_1.h5")
# define model
# model = Sequential()
# model.add(LSTM(50, activation='relu', input_shape=(128, 7)))
# model.add(Dense(1))
# model.compile(optimizer=Adam(clipnorm=1.), loss='mse')
# model.fit(X, y, epochs=20, verbose=1)
# model.save('daniels_lstm_mts_128_1.h5')

In [27]:
from tsx.xai.lime import LIMETimeSeries
import tqdm
def predict_fn(z, model=model):
    z_reshaped = z.T.reshape(1, 128, 7)
    z_hat = model.predict(z_reshaped)
    # to avoid zero coef_ for z_hat in[0, 1]
    z_hat = y_scaler.inverse_transform(z_hat.reshape(-1, 1))  
    z_hat = z_hat.ravel()   # z_hat will arround 50 - 150
    return z_hat[0]

X_lime = []
X_random = []

N=100
for i in tqdm.tqdm(range(N)):
    # create local lime explanation
    sample = np.array(X_test[i], copy=True)
    ts_x = sample.T # (n_features, n_steps) 
    ts_lime = LIMETimeSeries(scale="sync", window_size=1, sample_size=100)
    ts_lime = ts_lime.explain(ts_x, predict_fn=predict_fn)

    # convert coef to original shape of ts_x
    x_coef = ts_lime.perturb_obj._x_masked(ts_x, ts_lime.coef)
    
    # create random values
    # r = np.random.rand(*sample.shape)
    r = np.zeros_like(sample)

    # find the most important values and replace them with random values
    important_coef = ts_lime.coef[ts_lime.coef != 0]
    mask = np.ma.masked_where(np.logical_or(x_coef < np.percentile(important_coef, 10), 
                                            x_coef > np.percentile(important_coef, 90))
                              , x_coef).mask
    mask = mask.reshape(n_variables, -1).T
    sample[mask] = r[mask]
    X_lime.append(sample)

    # shuffle the mask to randomly change the same amount of features
    sample = np.array(X_test[i], copy=True)
    np.random.shuffle(mask)
    sample[mask] = r[mask]
    X_random.append(sample)

100%|██████████| 100/100 [12:02<00:00,  7.23s/it]


In [28]:

from sklearn.metrics import mean_squared_error
X_lime = np.array(X_lime)
X_random = np.array(X_random)
predictions_test = model.predict(X_test[:N])
predictions_random = model.predict(X_random)
predictions_lime = model.predict(X_lime)
test_subset = y_test[:N]

print(f"MSE {mean_squared_error(predictions_test, test_subset):.4f}, MSE (random) {mean_squared_error(predictions_random, test_subset):.4f}, MSE (LIME) {mean_squared_error(predictions_lime, test_subset):.4f}")
print(f"MSE vs Random {mean_squared_error(predictions_test, test_subset) - mean_squared_error(predictions_random, test_subset):.4f}")
print(f"MSE vs LIME {mean_squared_error(predictions_test, test_subset) - mean_squared_error(predictions_lime, test_subset):.4f}")
print(f"Random vs LIME {mean_squared_error(predictions_random, test_subset) - mean_squared_error(predictions_lime, test_subset):.4f}")

MSE 0.0032, MSE (random) 0.0033, MSE (LIME) 0.0056
MSE vs Random -0.0000
MSE vs LIME -0.0023
Random vs LIME -0.0023


### Attemp: (scale=async, wsize=1, r=random)
MSE 0.0032, MSE (random) 0.0045, MSE (LIME) 0.0068

- MSE vs Random -0.0013
- MSE vs LIME -0.0036
- Random vs LIME -0.0023

### Attemp: (scale=async, wsize=1, r=zeros)
MSE 0.0032, MSE (random) 0.0034, MSE (LIME) 0.0136

- MSE vs Random -0.0001
- MSE vs LIME -0.0104
- Random vs LIME -0.0103

### Attemp: (scale=async, wsize=8, r=zeros)
MSE 0.0032, MSE (random) 0.0057, MSE (LIME) 0.0154

- MSE vs Random -0.0025
- MSE vs LIME -0.0121
- Random vs LIME -0.0096


### Attemp: (scale=sync, wsize=8, r=zeros)
MSE 0.0032, MSE (random) 0.0037, MSE (LIME) 0.0110

- MSE vs Random -0.0005
- MSE vs LIME -0.0077
- Random vs LIME -0.0073

### Attemp: (scale=sync, wsize=1, r=zeros)
MSE 0.0032, MSE (random) 0.0033, MSE (LIME) 0.0056

- MSE vs Random -0.0000
- MSE vs LIME -0.0023
- Random vs LIME -0.0023

In [ ]:
for i in [ts_lime, predictions_lime, predictions_random, predictions_test, X_lime, X_random, x_coef, ts_x, mask, sample, ]:
    del i